## balanced_profolio

In [15]:
import pandas as pd
import numpy as np
import math

In [51]:
path = '/Users/yutingmei/APlusBernstein-Project/data/pred/lstm_preds.csv'
target = pd.read_csv(path)

In [52]:
target = target.iloc[1: , :]
target = target.reset_index(drop=True)

In [53]:
target

,sc,mom,value,.pred_prob_sc_lstm,.pred_prob_mom_lstm,.pred_prob_value_lstm,.pred_sc_lstm,.pred_mom_lstm,.pred_value_lstm,s_&_p_500_ret,sc_1d_fwd_ret,mom_1d_fwd_ret,value_1d_fwd_ret
0,0,0,1,0.507404,0.537511,0.537511,1,1,0,-0.000333,-0.008109,-0.002763,-0.001990
1,1,1,1,0.513883,0.548627,0.548627,1,1,0,-0.002512,0.008351,0.009081,0.006174
2,1,1,1,0.525089,0.551225,0.551225,1,1,0,0.006082,0.001307,0.007427,0.001928
3,0,1,1,0.535904,0.556041,0.556041,1,1,0,-0.000212,0.000261,0.001359,0.002828
4,1,1,0,0.517375,0.554009,0.554009,1,1,0,0.000348,0.005483,0.003284,0.000307
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1,0,0,0.473095,0.508067,0.508067,0,1,0,-0.000215,0.001707,-0.004040,-0.002439
1841,0,0,0,0.475162,0.510649,0.510649,0,1,0,-0.000846,-0.003714,-0.004661,0.006674
1842,0,0,0,0.484604,0.517438,0.517438,0,1,0,0.006763,-0.013597,-0.007605,-0.009775
1843,1,0,1,0.492156,0.528105,0.528105,0,1,0,-0.007195,0.004891,-0.007206,0.008705


In [38]:
#path = '/Users/yutingmei/Projects/APlusBernstein-Project/data/raw/FactorTimingData-ABVU2022.xlsx'
#target = pd.read_excel(path, sheet_name = 'target')

In [55]:
#chceck this code I think there is something wrong with s&p data
def balanced_profolio(target):
    #target_ = target.iloc[:, 1:4]
    # target = target.iloc[:, -3:]#the return values
    start = 100000
    #s_p = target.iloc[:, 4]
    s_p = target.iloc[:, 9]#s&p
    #s_p = target["s_&_p_500_ret"]
    real = target.iloc[:, 0:3]#real values in 1 or 0
    compare_result = []
    pred = target.iloc[:,7:10]
    op = target.iloc[:,-3:]
    for i in range(target.shape[0]):
        row = op.iloc[i, :]
        add = sum(list(map(lambda x: (start / 3) * (x), row)))
        if pred.iloc[i,:].sum() >= 2:
            # pred direction
            op_invest = 1
            op_s_p = -1
        else:
            op_invest = -1
            op_s_p = 1
        if real.iloc[i, :].sum() >= 2:
            # real direction
            op_real = 1
        else:
            op_real = -1
        if op_s_p != op_invest:
            op_final = -1
        else:
            op_final = 1
        initial = start
        start = (op_invest * add + op_s_p * s_p[i] * initial) * op_final
        compare_result.append(start)
    return compare_result

In [56]:
balanced_profolio(target)

[-395.40479746328003,
 -4.104575460947678,
 0.010375145027130106,
 1.7586555563293423e-05,
 4.7070235956965286e-08,
 -7.69475707995297e-10,
 -1.9382214440959264e-11,
 1.3140045364123303e-13,
 -8.258500679646146e-16,
 2.0018098039152377e-18,
 1.7581556529176042e-20,
 2.3112331205174773e-23,
 -2.3574334376940203e-25,
 3.9360577327752804e-27,
 4.3740199412843846e-29,
 6.622685636818763e-31,
 -1.235644406548933e-32,
 -3.1837500677648567e-34,
 5.7579774852908984e-36,
 -1.165419562221196e-37,
 -3.514096498711389e-39,
 3.897447259910985e-41,
 5.042002712793172e-43,
 1.0928178072275205e-45,
 -1.3525812762927496e-47,
 -1.3253945633829668e-49,
 1.2543992401649646e-51,
 1.1848049076863159e-53,
 -2.498782554467989e-56,
 -3.05609125051496e-59,
 2.8410506000263414e-61,
 4.5608280645970923e-63,
 -2.7633210595363834e-65,
 -2.7513759388083066e-67,
 1.9001764183292548e-69,
 7.133059854877729e-72,
 3.3323775725792827e-74,
 -1.8410657778171785e-76,
 1.6784564465394647e-78,
 4.554569561576574e-80,
 -6.2515

In [51]:
#pred = target.iloc[:, 6:9]
#balanced_profolio(target, pred = target.iloc[:, 6:9])

In [58]:
#understanding s&p trend 
#this is invest in s&p and let it go for the entire time period 
#let the s&p run for the entire trend our model should do better than that 
#here its previous start*s&p and then that value is multiplied to next s&p and so on
s_p = target.iloc[:, 9]
#s_p = s_p.iloc[1: , :]

In [ ]:
start = 100000
value = 0
l = []
for i in s_p:
    start = i*start
    l.append(start)
    start == start